# Kütüphaneleri içe aktarma

In [ ]:
# Gerekli kütüphanelerin yüklenmesi
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, GlobalMaxPooling1D, Dropout, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
"""
Bu kod bloğu, derin öğrenme modeli oluşturmak ve IMDB veri setini işlemek için gerekli kütüphaneleri import eder.

Imports:
    tensorflow.keras: Derin öğrenme modellerini oluşturmak ve eğitmek için yüksek seviyeli API sağlar.
    tensorflow.keras.datasets.imdb: IMDB film yorumları veri setini yüklemek için kullanılır.
    tensorflow.keras.models.Sequential: Katmanları sıralı bir şekilde ekleyerek model oluşturmayı sağlar.
    tensorflow.keras.layers.Dense, Embedding, Flatten: Model mimarisinde kullanılacak katman türlerini içerir.
    tensorflow.keras.preprocessing.sequence.pad_sequences: Dizileri eşit uzunlukta hale getirmek için kullanılır.

Returns:
    None

Not:
    Bu importlar, bir duygu analizi modeli oluşturmak için IMDB veri setini kullanacak bir projenin temelini oluşturur.
"""
print("Gerekli kütüphaneler başarıyla yüklendi.")


# Veri kümesi işlemleri

In [ ]:
# Veri setinin yüklenmesi
"""
Bu kod bloğu, IMDB film yorumları veri setini yükler ve eğitim ve test verilerine ayırır.

Args:
    num_words (int): Veri setinde kullanılacak en sık kullanılan kelime sayısı

Returns:
    tuple: İki tuple içeren bir tuple:
        - (X_train, y_train): Eğitim verileri ve etiketleri
        - (X_test, y_test): Test verileri ve etiketleri

X_train ve X_test: Film yorumlarının kelime dizileri (liste olarak)
y_train ve y_test: Yorumların duygu etiketleri (0: negatif, 1: pozitif)

Not:
    Bu fonksiyon, veri setini otomatik olarak eğitim ve test setlerine ayırır.
    num_words parametresi, veri setinde kullanılacak en sık kelime sayısını sınırlar.
"""
# veri kümesindeki en büyük indeksi bulma (kelime sayısı)
max_index = 10000
print("IMDB veri kümesi yükleniyor...")
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_index)
print("Veri kümesi başarıyla yüklendi.")

In [ ]:
# Veri kümesinin incelenmesi
"""
Bu kod bloğu, yüklenen IMDB veri kümesinin temel özelliklerini inceler ve gösterir.

Args:
    None

Returns:
    None

Çıktı:
    - Eğitim veri seti boyutu
    - Test veri seti boyutu
    - İlk eğitim örneğinin içeriği

Not:
    Bu inceleme, veri kümesinin yapısını ve boyutunu anlamak için önemlidir.
    İlk eğitim örneği, bir film yorumunun nasıl temsil edildiğini gösterir.
"""

print(f"Eğitim veri seti boyutu: {len(X_train)}")
print(f"Test veri seti boyutu: {len(X_test)}")
print(f"İlk eğitim örneği:")
print(X_train[0])

In [ ]:
# Veri kümesindeki kelimeleri kelime dağarcığına çevirme
"""
Bu kod bloğu, IMDB veri kümesindeki kelimelerin indekslerini içeren bir sözlük yükler.

word_index (dict): Kelimeleri ve karşılık gelen indekslerini içeren bir sözlük

Not:
    Bu sözlük, kelimeler ve onların sayısal temsilleri arasında bir eşleştirme sağlar.
    Bu, daha sonra film yorumlarındaki sayısal indeksleri anlamlı kelimelere çevirmek için kullanılabilir.
"""
word_index = imdb.get_word_index()
print("Kelime indeksi yüklendi.")

In [ ]:
# Ters çevirme işlemi ile indeksleri kelimelere dönüştürme
"""
Bu kod bloğu, sayısal indeksleri kelimelere dönüştürmek için bir sözlük ve bir fonksiyon oluşturur.

reverse_word_index (dict): İndeksleri kelimelere eşleyen ters çevrilmiş sözlük
decode_review (function): Sayısal indeks listesini okunabilir metne dönüştüren fonksiyon

Not:
    - reverse_word_index, word_index sözlüğünün ters çevrilmiş halidir.
    - decode_review fonksiyonu, bir film yorumunu sayısal indekslerden okunabilir metne çevirir.
    - İndekslerden 3 çıkarılır çünkü IMDB veri setinde 0, 1, 2 indeksleri özel anlamlara sahiptir.
"""
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in text])
ilk_eleman = next(iter(reverse_word_index.items()))
print(ilk_eleman)

In [ ]:
"""
Bu kod bloğu, eğitim setindeki ilk film yorumunu okunabilir formatta gösterir.
Çıktı:
    İlk eğitim örneğinin okunabilir metin hali

Not:
    - Bu, daha önce tanımlanan decode_review fonksiyonunu kullanır.
    - X_train[0], eğitim setindeki ilk film yorumunun sayısal indeks listesidir.
    - Çıktı, bu sayısal indekslerin kelimelere dönüştürülmüş halidir.
"""
print("İlk eğitim örneğinin orijinal hali:")
print(decode_review(X_train[0]))

In [14]:
# Eğitim verisi olan X_train'i makine öğrenmesi modeline uygun hale getirmek için ön işleme tabi tutuyoruz.
# Doğal dil işleme uygulamalarında cümleler veya kelime dizileri genellikle farklı uzunluklarda olabilir.
# Ancak sinir ağları genellikle sabit boyutlu girdi verilerine ihtiyaç duyar.
# Bu nedenle, farklı uzunluklardaki dizileri aynı uzunluğa getirmek için sıfırlama (padding) işlemi yapıyoruz.

# pad_sequences fonksiyonu, dizilerin uzunluğunu eşitlemek için kullanılır.
# padding='post' parametresi ile, eksik olan öğeleri dizinin SONUNA ekleyerek eşitliyoruz.
# Böylece orijinal dizideki verilerin sırası bozulmaz ve veri kaybı olmaz.
# Sıfırlama işlemi sonucunda, tüm diziler aynı uzunlukta olup model tarafından işlenebilir hale gelir.

X_train = pad_sequences(X_train, padding='post')

# Test verisi olan X_test'i de aynı şekilde işliyoruz.
# Bu, modelin eğitim ve test aşamasında tutarlı ve uyumlu verilerle çalışmasını sağlar.
# Aynı sıfırlama işlemi uygulanmazsa, model test verisini doğru bir şekilde değerlendiremeyebilir.

X_test = pad_sequences(X_test, padding='post')

# Model oluşturma

In [ ]:
# LSTM Modeli'nin oluşturulması
"""
Bu kod bloğu, duygu analizi için bir sinir LSTM modeli oluşturur.

lstm_model (Sequential): Oluşturulan Keras Sequential model

Not:

    - Model, bir Embedding katmanı, iki LSTM katmanı, bir Dense katman ve bir Dropout katmanından oluşur.
    - Embedding katmanı, kelime indekslerini yoğun vektörlere dönüştürür.
    - LSTM katmanları, uzun vadeli bağımlılıkları modellemek için kullanılır.
    - Dense katman, son sınıflandırma işlemini gerçekleştirir.
    - 'sigmoid' aktivasyon fonksiyonu, çıktıyı 0 ile 1 arasında bir olasılığa dönüştürür.
    - 'adam' optimizer, modelin eğitimini gerçekleştirir.
    - 'binary_crossentropy' loss fonksiyonu, modelin başarısını ölçer.
    - 'accuracy' metriği, modelin doğruluğunu ölçer.
"""

print("LSTM Modeli oluşturuluyor...")
lstm_model = Sequential()
lstm_model.add(Embedding(max_index, 32))
lstm_model.add(LSTM(64, return_sequences=True))
lstm_model.add(LSTM(32))
lstm_model.add(Dense(16, activation='relu'))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(lstm_model.summary())

In [ ]:
# CNN Modeli'nin oluşturulması
"""
Bu kod bloğu, duygu analizi için bir sinir ağı CNN modeli oluşturur.

cnn_model (Sequential): Oluşturulan Keras Sequential model

Not:
    
        - Model, bir Embedding katmanı, iki Conv1D katmanı, bir GlobalMaxPooling1D katmanı, iki Dense katman ve bir Dropout katmanından oluşur.
        - Embedding katmanı, kelime indekslerini yoğun vektörlere dönüştürür.
        - Conv1D katmanları, metin verilerindeki desenleri öğrenmek için kullanılır.
        - GlobalMaxPooling1D katmanı, Conv1D katmanlarının çıktılarını düzleştirir.
        - Dense katmanlar, sınıflandırma işlemini gerçekleştirir.
        - 'sigmoid' aktivasyon fonksiyonu, çıktıyı 0 ile 1 arasında bir olasılığa dönüştürür.
        - 'adam' optimizer, modelin eğitimini gerçekleştirir.
        - 'binary_crossentropy' loss fonksiyonu, modelin başarısını ölçer.
        - 'accuracy' metriği, modelin doğruluğunu ölçer.
"""

print("CNN Modeli oluşturuluyor...")
cnn_model = Sequential()
cnn_model.add(Embedding(max_index, 32))
cnn_model.add(Conv1D(64, 5, activation='relu'))
cnn_model.add(Conv1D(32, 3, activation='relu'))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(16, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(1, activation='sigmoid'))

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(cnn_model.summary())

In [ ]:
"""
Bu kod bloğu, oluşturulan modelin özetini gösterir.
Çıktı:
    Modelin katmanları, parametreleri ve toplam parametre sayısı hakkında detaylı bilgi

Not:
    - model.summary() fonksiyonu, modelin yapısını, her katmanın çıktı şeklini ve parametre sayısını gösterir.
    - Bu özet, modelin karmaşıklığını ve bellek gereksinimlerini anlamak için çok faydalıdır.
"""
print("LSTM Modeli:")
print(lstm_model.summary())

In [ ]:
print("CNN Modeli:")
print(cnn_model.summary())

# Model eğitme işlemleri

In [ ]:
# Modelin derlenmesi
"""
Bu kod bloğu, oluşturulan modeli derler ve eğitim için hazırlar.
Not:
    - model.compile() fonksiyonu, modelin eğitim sürecini yapılandırır.
    - optimizer='adam': Adam optimizasyon algoritması kullanılacak.
    - loss='binary_crossentropy': İkili sınıflandırma için uygun kayıp fonksiyonu.
    - metrics=['accuracy']: Eğitim sırasında doğruluk metriği izlenecek.
"""
print("Modeller derleniyor...")
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print("LSTM modeli derleme tamamlandı.")
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print("CNN modeli derleme tamamlandı.")
print("Modelleri derleme tamamlandı.")

In [ ]:
# Modelin eğitilmesi
"""
Bu kod bloğu, oluşturulan modeli eğitir.

X_train (numpy.ndarray): Eğitim verileri
y_train (numpy.ndarray): Eğitim etiketleri
history (keras.callbacks.History): Eğitim sürecinin geçmişi

Not:
    - model.fit() fonksiyonu, modeli verilen veriler üzerinde eğitir.
    - epochs=5: Model, veri seti üzerinde 5 kez tekrarlanacak.
    - batch_size=128: Her iterasyonda 128 örnek işlenecek.
    - validation_split=0.2: Eğitim verilerinin %20'si doğrulama için ayrılacak.
    - 1 epoch yaklaşık 3-4 dakika sürdü.
"""
print("Modeller eğitiliyor...")
print("LSTM Modeli eğitiliyor...")
lstm_history = lstm_model.fit(X_train, y_train, epochs=2, batch_size=256, validation_split=0.2)
print("LSTM Modeli eğitimi tamamlandı.")
print("CNN Modeli eğitiliyor...")
cnn_history = cnn_model.fit(X_train, y_train, epochs=2, batch_size=256, validation_split=0.2)
print("CNN Modeli eğitimi tamamlandı.")
print("Model eğitim tamamlandı.")
lstm_history.history, cnn_history.history

In [ ]:
# Modelin değerlendirilmesi

"""
Bu kod bloğu, eğitilmiş modeli test veri seti üzerinde değerlendirir.

Çıktı:
    - Test Loss: Modelin test veri seti üzerindeki kayıp değeri
    - Test Accuracy: Modelin test veri seti üzerindeki doğruluk oranı

Not:
    - model.evaluate() fonksiyonu, modeli verilen veri seti üzerinde değerlendirir.
    - Kayıp (loss) değeri, modelin tahminlerinin gerçek değerlerden ne kadar uzak olduğunu gösterir.
    - Doğruluk (accuracy) değeri, modelin doğru tahmin ettiği örneklerin oranını gösterir.
    - Yaklaşık 4 dakika sürer.
"""
print("Modeller test verisi üzerinde değerlendiriliyor...")
print("LSTM Modeli değerlendiriliyor...")
lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test, y_test)
print("LSTM Modeli değerlendirme tamamlandı.")
print("CNN Modeli değerlendiriliyor...")
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test, y_test)
print("CNN Modeli değerlendirme tamamlandı.")
print("Modeller değerlendirme tamamlandı.")
print(f"LSTM Modeli - Test Loss: {lstm_loss}, Test Accuracy: {lstm_accuracy}")
print(f"CNN Modeli - Test Loss: {cnn_loss}, Test Accuracy: {cnn_accuracy}")

# Modeli elle test etme

## Fonksiyon tanımları

In [20]:
def get_user_review():
    """
    Kullanıcıdan film yorumu alır.

    Returns:
        str: Kullanıcının girdiği film yorumu

    Not:
        - Kullanıcıya bir giriş istemi sunar.
        - 'q' girişi, programdan çıkış için kullanılır.
    """
    return input("Lütfen bir film yorumu girin (Çıkmak için 'q' yazın): ")

def preprocess_text(text):
    """
    Metni kelimelere ayırır.

    Args:
        text (str): İşlenecek metin

    Returns:
        list: Metindeki kelimelerin listesi

    Not:
        - Keras'ın text_to_word_sequence fonksiyonunu kullanır.
        - Bu fonksiyon, metni küçük harflere çevirir, noktalama işaretlerini kaldırır ve kelimelere ayırır.
    """
    return keras.preprocessing.text.text_to_word_sequence(text)

def tokenize_words(tokens):
    """
    Kelimeleri sayısal indekslere dönüştürür.

    Args:
        tokens (list): Kelime listesi

    Returns:
        list: Kelimelerin sayısal indeks karşılıkları

    Not:
        - word_index sözlüğünü kullanarak her kelimeyi bir sayıya dönüştürür.
        - Sözlükte olmayan kelimeler için 1 değerini kullanır (genellikle bilinmeyen kelime indeksi).
    """
    return [word_index.get(word, 1) for word in tokens]

def limit_token_range(tokens):
    """
    Token değerlerini 10000'in altında tutar.

    Args:
        tokens (list): Sayısal token listesi

    Returns:
        list: 10000'in altındaki değerlere sınırlandırılmış token listesi

    Not:
        - 10000'den büyük indeksleri 1'e çevirir.
        - Bu, modelin kelime dağarcığı boyutunu sınırlar ve aşırı büyük indeksleri engeller.
    """
    return [t if t < max_index else 1 for t in tokens]

def pad_tokens(tokens):
    """
    Token dizisini sabit uzunluğa getirir.

    Args:
        tokens (list): Sayısal token listesi

    Returns:
        numpy.ndarray: Sabit uzunluğa getirilmiş token dizisi

    Not:
        - Keras'ın pad_sequences fonksiyonunu kullanır.
        - maxlen değişkeni, dizinin hedef uzunluğunu belirtir.
        - Kısa diziler 0 ile doldurulur, uzun diziler kesilir.
    """
    return pad_sequences([tokens], maxlen=max_index)

def encode_review(text):
    """
    Yorumu model için uygun formata dönüştürür.

    Args:
        text (str): Orijinal yorum metni

    Returns:
        numpy.ndarray: Model için hazırlanmış, kodlanmış yorum

    Not:
        - Metni işlemek için önceki fonksiyonları sırayla çağırır.
        - Son çıktı, modelin beklediği formatta bir numpy dizisidir.
    """
    tokens = preprocess_text(text)
    tokens = tokenize_words(tokens)
    tokens = limit_token_range(tokens)
    return pad_tokens(tokens)

def predict_sentiment(encoded_review, model):
    """
    Kodlanmış yorum için duygu tahmini yapar.

    Args:
        encoded_review (numpy.ndarray): Kodlanmış yorum

    Returns:
        numpy.ndarray: Modelin tahmin sonucu

    Not:
        - Önceden eğitilmiş modeli kullanarak tahmin yapar.
        - Sonuç, yorumun pozitif olma olasılığını içerir.
    """
    return model.predict(encoded_review)

def print_prediction(review, lstm_prediction, cnn_prediction):
    """
    Tahmin sonucunu ekrana yazdırır.

    Args:
        review (str): Orijinal yorum metni
        lstm_prediction (numpy.ndarray): LSTM modelinin tahmin sonucu
        cnn_prediction (numpy.ndarray): CNN modelinin tahmin sonucu

    Returns:
        None

    Not:
        - Orijinal yorumu ve pozitif olma olasılığını yazdırır.
        - Olasılık 4 ondalık basamağa yuvarlanır.
    """
    print(f"\nYorum: {review}")
    print(f"LSTM Modeli Tahmini: %{lstm_prediction[0][0]*100:.4f}")
    print(f"CNN Modeli Tahmini: %{cnn_prediction[0][0]*100:.4f}")

## Elle test

In [ ]:
print("Film yorumları için duygu analizi uygulaması başlatılıyor...")
while True:
    review = get_user_review()
    if review.lower() == 'q':
        print("Uygulama sonlandırılıyor...")
        break
    
    encoded_review = encode_review(review)
    lstm_prediction = predict_sentiment(encoded_review, lstm_model)
    cnn_prediction = predict_sentiment(encoded_review, cnn_model)
    print_prediction(review, lstm_prediction, cnn_prediction)